In [ ]:
# !git clone https://github.com/ajay-sainy/Wav2Lip-GFPGAN.git
# basePath = "/content/Wav2Lip-GFPGAN"
# %cd {basePath}

In [62]:
import os
basePath = os.getcwd()+'/Wav2Lip'
wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
wav2lipPath = wav2lipFolderName
gfpganPath = gfpganFolderName

# No need to Run; Models already downloaded
# !wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'
# !gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}'/checkpoints/'

In [ ]:
!pip install -r requirements.txt

In [64]:
import os
basePath = os.getcwd()+"/Wav2Lip"
!ls

Output             RAG_Stripped       Wav2Lip            temp
RAG                VideoChatbot.ipynb outputs            test.ipynb


In [87]:
import os
basePath = os.getcwd()+'/Wav2Lip'
outputPath = 'Output/test'
inputAudioPath = basePath + '/inputs/kim_audio.mp3'
inputVideoPath = basePath + '/inputs/Palki.mp4'
lipSyncedOutputPath = 'Output/test/video/result_test.mp4'

if not os.path.exists(outputPath):
  os.makedirs(outputPath)

#!cd $basePath && cd $wav2lipFolderName 
!python Wav2Lip/Wav2Lip-master/inference.py \
--checkpoint_path Wav2Lip/Wav2Lip-master/checkpoints/wav2lip.pth \
--face {inputVideoPath} \
--audio {inputAudioPath} \
--outfile {lipSyncedOutputPath}

Using mps for inference.
Reading video frames...
Number of frames available for inference: 332
Extracting raw audio...
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.0.40.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libop

In [ ]:
#Download GFPGAN already downloaded
# !cd $gfpganFolderName && python setup.py develop
# !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P {gfpganFolderName}'/experiments/pretrained_models'

In [115]:
# Description: This script is used to generate frames from the lip-synced video
import cv2
from tqdm import tqdm
from os import path

import os
outputPath = 'Output/test'
inputVideoPath = outputPath+'/video/result_test.mp4'
unProcessedFramesFolderPath = outputPath+'/frames'

if not os.path.exists(unProcessedFramesFolderPath):
  os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)


  0%|          | 0/337 [00:00<?, ?it/s]

  7%|▋         | 23/337 [00:00<00:01, 221.87it/s]

FPS:  25.05956293742694 Frames:  337


 99%|█████████▊| 332/337 [00:01<00:00, 209.02it/s]


error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp:786: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


In [ ]:
# # This command is used to enhance the lipsync video frames using GFPGAN
# video_enhance_cmd = ["python",
#                      "Wav2Lip/GFPGAN-master/inference_gfpgan.py", 
#                      "-i", unProcessedFramesFolderPath,
#                      "-o", outputPath 
#                      "-v", 1.3 "-s" 2 ,
#                      "--only_center_face", "--bg_upsampler", None
#                      ]

# import subprocess
# subprocess.run(video_enhance_cmd)

In [109]:
!python Wav2Lip/GFPGAN-master/inference_gfpgan.py -i $unProcessedFramesFolderPath -o $outputPath -v 1.3 -s 2 --only_center_face --bg_upsampler None -device 'mps'

/Users/shawn/anaconda3/envs/llamalocal3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/shawn/anaconda3/envs/llamalocal3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|█████████████████████████████████████████| 183/183 [01:48<00:00,  1.69it/s]
Results are in the [Output/test] folder.


In [113]:
# This scrpit is used to generate the video from the enhanced frames
import os
restoredFramesPath = outputPath + '/restored_imgs/'
processedVideoOutputPath = outputPath

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2

batch = 0
batchSize = 100
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end)
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)


  out = cv2.VideoWriter(processedVideoOutputPath+'/batch_'+str(batch).zfill(4)+'.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 25, size)
  batch = batch + 1
 
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

  0%|          | 0/2 [00:00<?, ?it/s]

processing  0 100


 50%|█████     | 1/2 [00:01<00:01,  1.91s/it]

processing  100 200


100%|██████████| 2/2 [00:03<00:00,  1.72s/it]


In [ ]:
# Concatenate the batch videos into a single video using ffmpeg library.
concatTextFilePath = outputPath + "/concat.txt"
concatTextFile=open(concatTextFilePath,"w")
for ips in range(batch):
  concatTextFile.write("file batch_" + str(ips).zfill(4) + ".mp4\n")
concatTextFile.close()

concatedVideoOutputPath = outputPath + "/concated_output.mp4"
import_videos_command =["ffmpeg", "-y", "-f", "concat", 
          "-i", concatTextFilePath, "-c", 
          "copy", concatedVideoOutputPath 
        ]

finalProcessedOuputVideo = processedVideoOutputPath+'/final_with_audio.mp4'
concat_video_command =["ffmpeg", "-y", "-i", concatedVideoOutputPath,
                       "-i", inputAudioPath, "-map", "0", "-map", "1:a", 
                       "-c:v", "copy", "-shortest",finalProcessedOuputVideo
                      ]

import subprocess
subprocess.run(import_videos_command)
subprocess.run(concat_video_command)

In [114]:
# Concatenate the batch videos into a single video using ffmpeg library.
concatTextFilePath = outputPath + "/concat.txt"
concatTextFile=open(concatTextFilePath,"w")
for ips in range(batch):
  concatTextFile.write("file batch_" + str(ips).zfill(4) + ".mp4\n")
concatTextFile.close()

concatedVideoOutputPath = outputPath + "/concated_output.mp4"
!ffmpeg -y -f concat -i {concatTextFilePath} -c copy {concatedVideoOutputPath} 

finalProcessedOuputVideo = processedVideoOutputPath+'/final_with_audio.mp4'
!ffmpeg -y -i {concatedVideoOutputPath} -i {inputAudioPath} -map 0 -map 1:a -c:v copy -shortest {finalProcessedOuputVideo}

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.0.40.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enabl